In [ ]:
import keras
import tensorflow
import h5py
import numpy as np
import chowdhury_s_model_builder
import os

import tensorflow as tf
from keras.api.models import Sequential, Model, load_model
from keras.api.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization, LSTM, Input, GlobalAveragePooling2D
from keras.api.optimizers import Adam
from keras import regularizers

import tensorflow as tf
print(tf.__version__)

from sklearn.metrics import classification_report, multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [ ]:
with h5py.File("Data/Emotify_Annotated_Data.h5", "r") as hf:
    train_data = hf["train"][:]  # Load the train dataset
    label_data = hf["label"][:]  # Load the label dataset

print(train_data.shape, train_data.dtype)  # Check the shape and type
print(label_data.shape, label_data.dtype)

In [ ]:
train_data = train_data[..., np.newaxis]

print(train_data.shape, train_data.dtype)  # Check the shape and type
print(label_data.shape, label_data.dtype)

In [ ]:
input_shape = train_data.shape[1], train_data.shape[2], train_data.shape[3]
print(input_shape)

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming train_data and label_data are already loaded
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_data, label_data, test_size=0.2, random_state=42)

# Optionally, split the training set further into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Print the shapes to verify
print("Training set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Test set shape:", X_test.shape, y_test.shape)

In [ ]:
model = load_model("model_epoch_86_acc_0.94_loss_0.01.keras")
model.summary()

In [ ]:
model.compile(optimizer=Adam(learning_rate=3.1250e-05),
                           loss='binary_crossentropy', metrics=["accuracy"]
                           )

In [ ]:
results = model.evaluate(train_data, label_data, batch_size=8, verbose=1)
print("Validation Loss:", results[0])
print("Validation Accuracy:", results[1])
# Predict the labels for the validation set

In [ ]:
# Assuming 'model' is your trained model
y_pred = model.predict(train_data)
# Convert probabilities to binary labels (e.g., threshold at 0.5)
y_pred_binary = (y_pred > 0.5).astype(int)

In [ ]:

label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']
# Generate the classification report
report = classification_report(label_data, y_pred_binary, target_names=label_names)
print(report)

# Generate the multilabel confusion matrix
mcm = multilabel_confusion_matrix(label_data, y_pred_binary)
print("Multilabel Confusion Matrix:")
print(mcm)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define label names
label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']

# Plot the confusion matrix for each label
for i, cm in enumerate(mcm):
    print(f'Confusion Matrix for {label_names[i]}')
    print('Actual', cm)
    print("Predicted")
    # plt.figure(figsize=(6, 4))
    # sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
    # plt.title(f'Confusion Matrix for {label_names[i]}')
    # plt.xlabel('Predicted')
    # plt.ylabel('Actual')
    # plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
# import ace_tools as tools
# Define label names
label_names = ['melody', 'articulation', 'rhythm_complexity', 'rhythm_stability', 'dissonance', 'atonality', 'mode']

metrics = []

for i, cm in enumerate(mcm):
    tn, fp = cm[0]
    fn, tp = cm[1]
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (tp + tn) / cm.sum()
    metrics.append({
        'label': label_names[i],
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'accuracy': accuracy
    })

df_metrics = pd.DataFrame(metrics)
# Display metrics table
print(df_metrics)
# tools.display_dataframe_to_user(name="Label-wise Metrics", dataframe=df_metrics)

# # Plot all confusion matrices in a grid
# fig, axes = plt.subplots(2, 3, figsize=(15, 10))
# axes = axes.flatten()
# for i, cm in enumerate(cms):
#     sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
#                 xticklabels=['Pred Neg', 'Pred Pos'],
#                 yticklabels=['True Neg', 'True Pos'],
#                 ax=axes[i])
#     axes[i].set_title(label_names[i])
#     axes[i].set_xlabel('Predicted')
#     axes[i].set_ylabel('Actual')

# plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

thresholds = [0.3, 0.4, 0.5, 0.6]
for t in thresholds:
    preds = (model.predict(train_data) > t).astype(int)
    precision = precision_score(label_data, preds, average='macro')
    recall = recall_score(label_data, preds, average='macro')
    f1 = f1_score(label_data, preds, average='macro')
    accuracy = accuracy_score(label_data, preds)
    print(f"\nThreshold: {t}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"  Accuracy:  {accuracy:.4f}")
